### Task Schedule:
1. 訓練模型,調整參數(預計使用lgb，速度較快)(更:使用catboost,效果較好)
2. 嘗試使用不同模型,做Ensamble(blending, stacking)
3. Anomaly detection

### 注意事項:
1. 因為test data和train data時間不相關,在驗證時採取前60天訓練61~90天驗證,但仍需小心時間差異造成的影響
2. Anomaly detection: 看這類的模型能不能取代boosting(似乎是不行，盜刷數據並沒有那麼Anomaly）,但可以嘗試將Anomaly結果當成新feature

### <font color=green>Results:</font>

#### Catboost:
    * FE1~4,catboost訓練 validation:0.5, LB:0.55
    * FE1,3,4 validation:0.5313149, LB:0.6(th=0.37), default parameter

#### LGB:
    * 不做處理,直接丟lgb訓練 leaderboard score:0.45
    # 585
    # 593
    # 不同的thread不同的結果 乾==

## 讀取,轉換字串成可以訓練的資料

In [72]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import math

import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier, Pool

import xgboost as xgb
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn import svm
from sklearn.ensemble  import  GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 

%matplotlib inline
data_path = '../../data'

random_seed = 20
import json
path = '../../code/para_dict/data_list.json'
with open(path,'r',encoding='utf-8') as f:
    para = json.loads(f.read())
    
data_list= para['data_list_FE_AN3']
delete_list = para['delete_list_overfit1']

## 除掉一些可能會overfit,distribution不同,受時間影響大的feature

In [73]:
def load_data(data_list):
    data=[]
    for d in data_list:
        x = pd.read_csv('../../data/preprocess/{}'.format(d))
        x_null = x.isnull().sum()
        
        print('\n',d,x.shape)
        print("Null columns:\n",x_null[x_null>0])

        if (d=='FE_data1.csv') or (d=='FE_data2.csv'):
            x.fillna(value=-1,inplace=True)

        data.append(x)

    all_data = pd.concat(data,axis=1)
    all_data_numsum = all_data.isnull().sum()
    print('ALL data shape:',all_data.shape)
    print('ALL data null:')
    print(all_data_numsum[all_data_numsum>0])
    return all_data

all_data = load_data(data_list)


 raw_data.csv (1943452, 23)
Null columns:
 fraud_ind    421665
dtype: int64

 FE_data1.csv (1943452, 56)
Null columns:
 cano_conam_skew      92612
cano_conam_kurt     155720
cano_conam_var       38678
bacno_locdt_skew     58303
bacno_locdt_kurt    101191
cano_locdt_skew      92612
cano_locdt_kurt     155720
dtype: int64

 FE_data2.csv (1943452, 30)
Null columns:
 Series([], dtype: int64)

 FE_data2_2.csv (1943452, 31)
Null columns:
 Series([], dtype: int64)

 FE_data3.csv (1943452, 9)
Null columns:
 Series([], dtype: int64)

 FE_data4.csv (1943452, 4)
Null columns:
 Series([], dtype: int64)

 FE_data4_2.csv (1943452, 3)
Null columns:
 Series([], dtype: int64)

 FE_data5.csv (1943452, 4)
Null columns:
 Series([], dtype: int64)

 FE_data6.csv (1943452, 17)
Null columns:
 Series([], dtype: int64)

 FE_data7.csv (1943452, 33)
Null columns:
 Series([], dtype: int64)

 FE_data7_2.csv (1943452, 33)
Null columns:
 Series([], dtype: int64)

 pca_feature.csv (1943452, 5)
Null columns:
 Series([

In [74]:
import sys
print("Memory size of "+str(sys.getsizeof(all_data))+ " bytes")

Memory size of 3848035064 bytes


In [77]:
category_list=['csmcu','hcefg','stscd','scity','stocn','mcc','acqic','mchno','etymd','contp','locdt_week']
category_list=[]#None

for c in category_list:       
    all_data[c]=all_data[c].astype('category')
for c in all_data.columns[all_data.dtypes==bool]:
    all_data[c]=all_data[c].map({True:1,False:0})

In [78]:
## 切三種不同的訓練集驗證
X_train1 = all_data[all_data['locdt']<=60].drop(columns=delete_list)
y_train1 = all_data[all_data['locdt']<=60]['fraud_ind']
X_test1 = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test1 = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)]['fraud_ind']

test_data_txkey = all_data[all_data['locdt']>90]['txkey'].copy().values
X_train_all = all_data[all_data['locdt']<=90].drop(columns=delete_list) 
y_train_all = all_data[all_data['locdt']<=90]['fraud_ind'] 

X_test_all = all_data[all_data['locdt']>90].drop(columns=delete_list) 
# y_test_all = all_data[all_data['locdt']>90]['fraud_ind'] 

## Train on catboost
* https://catboost.ai/docs/concepts/python-reference_parameters-list.html
* 研究有哪些可以用的function

In [79]:
categorical_features_indices = np.where(X_train1.columns.isin(category_list))[0]
print(X_train1.dtypes[categorical_features_indices])

Series([], dtype: object)


In [80]:
param_cat={
    'loss_function':'Logloss',
    'eval_metric':'F1',
    
    'iterations':200,
    'learning_rate':0.1,
    'l2_leaf_reg':3,
    'bagging_temperature':1,
#     'sampling_frequency':'PerTreeLevel',
    
    'depth':6,
    'one_hot_max_size':300,
    
#     'min_data_in_leaf':1,
#     'max_leaves':31,
#     'task_type':"GPU",
#     'devices':1',
#     'cat_features':categorical_features_indices,
    'thread_count':2,
    'rsm':1,
    'scale_pos_weight':1,
    'target_border':0.5,
    'random_seed':random_seed,
    'verbose':True    
}

'random_strength':1,
'depth':6,
'grow_policy':'Lossguide',
'min_data_in_leaf':1,
# 'max_leaves':31,
'boosting_type':'Ordered',
'gpu_ram_part':0.5,
    
# 將binary的資料轉成category


In [81]:
model = CatBoostClassifier(**param_cat)

model.fit(
    X_train1, y_train1,
    cat_features=categorical_features_indices,    
    eval_set=(X_test1, y_test1),
    early_stopping_rounds=300,
#     use_best_model=True,
    silent=False,
#     plot=True,
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

0:	learn: 0.5218418	test: 0.2561949	best: 0.2561949 (0)	total: 912ms	remaining: 3m 1s
1:	learn: 0.5413011	test: 0.3018664	best: 0.3018664 (1)	total: 1.42s	remaining: 2m 20s
2:	learn: 0.5601541	test: 0.3185651	best: 0.3185651 (2)	total: 1.86s	remaining: 2m 2s
3:	learn: 0.5882546	test: 0.3337717	best: 0.3337717 (3)	total: 2.29s	remaining: 1m 52s
4:	learn: 0.5955581	test: 0.3395732	best: 0.3395732 (4)	total: 2.76s	remaining: 1m 47s
5:	learn: 0.6013421	test: 0.3409386	best: 0.3409386 (5)	total: 3.23s	remaining: 1m 44s
6:	learn: 0.6031426	test: 0.3514623	best: 0.3514623 (6)	total: 3.64s	remaining: 1m 40s
7:	learn: 0.6068506	test: 0.3567692	best: 0.3567692 (7)	total: 4.13s	remaining: 1m 39s
8:	learn: 0.6152743	test: 0.3603785	best: 0.3603785 (8)	total: 4.57s	remaining: 1m 37s
9:	learn: 0.6161046	test: 0.3622660	best: 0.3622660 (9)	total: 4.99s	remaining: 1m 34s
10:	learn: 0.6250904	test: 0.3763066	best: 0.3763066 (10)	total: 5.39s	remaining: 1m 32s
11:	learn: 0.6171885	test: 0.3906436	best: 

93:	learn: 0.7469861	test: 0.5112669	best: 0.5112895 (83)	total: 47.8s	remaining: 53.9s
94:	learn: 0.7476324	test: 0.5107986	best: 0.5112895 (83)	total: 48.4s	remaining: 53.5s
95:	learn: 0.7475765	test: 0.5116688	best: 0.5116688 (95)	total: 48.9s	remaining: 53s
96:	learn: 0.7479354	test: 0.5128462	best: 0.5128462 (96)	total: 49.4s	remaining: 52.5s
97:	learn: 0.7485513	test: 0.5130391	best: 0.5130391 (97)	total: 49.8s	remaining: 51.9s
98:	learn: 0.7495931	test: 0.5147151	best: 0.5147151 (98)	total: 50.4s	remaining: 51.4s
99:	learn: 0.7504259	test: 0.5183796	best: 0.5183796 (99)	total: 50.9s	remaining: 50.9s
100:	learn: 0.7511357	test: 0.5188149	best: 0.5188149 (100)	total: 51.5s	remaining: 50.4s
101:	learn: 0.7514285	test: 0.5182409	best: 0.5188149 (100)	total: 52s	remaining: 50s
102:	learn: 0.7520702	test: 0.5185370	best: 0.5188149 (100)	total: 52.5s	remaining: 49.5s
103:	learn: 0.7528247	test: 0.5201246	best: 0.5201246 (103)	total: 53s	remaining: 48.9s
104:	learn: 0.7538311	test: 0.52

184:	learn: 0.7897443	test: 0.5413958	best: 0.5422898 (182)	total: 1m 34s	remaining: 7.66s
185:	learn: 0.7902176	test: 0.5409514	best: 0.5422898 (182)	total: 1m 34s	remaining: 7.15s
186:	learn: 0.7908577	test: 0.5409052	best: 0.5422898 (182)	total: 1m 35s	remaining: 6.64s
187:	learn: 0.7909162	test: 0.5409052	best: 0.5422898 (182)	total: 1m 36s	remaining: 6.14s
188:	learn: 0.7909333	test: 0.5418200	best: 0.5422898 (182)	total: 1m 36s	remaining: 5.62s
189:	learn: 0.7913510	test: 0.5414161	best: 0.5422898 (182)	total: 1m 37s	remaining: 5.11s
190:	learn: 0.7919131	test: 0.5425976	best: 0.5425976 (190)	total: 1m 37s	remaining: 4.6s
191:	learn: 0.7922270	test: 0.5424187	best: 0.5425976 (190)	total: 1m 38s	remaining: 4.09s
192:	learn: 0.7922409	test: 0.5431341	best: 0.5431341 (192)	total: 1m 38s	remaining: 3.58s
193:	learn: 0.7924333	test: 0.5434277	best: 0.5434277 (193)	total: 1m 39s	remaining: 3.07s
194:	learn: 0.7936227	test: 0.5448326	best: 0.5448326 (194)	total: 1m 39s	remaining: 2.56s


In [82]:
feature_names = X_test1.columns
validation_pool=Pool(X_test1, y_test1,cat_features=categorical_features_indices)

# PredictionValuesChange
feature_importances = model.get_feature_importance(validation_pool)
print('PredictionValuesChange')
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score)) 

PredictionValuesChange
isolationtree_all_feature3: 8.482123054163972
stocn_cano_nunique: 8.410840326461201
stocn_bacno_nunique: 6.48823376186886
contp_value_counts_cano: 6.052415024854486
ecfg: 3.930377648455835
bacno_1_count: 3.3135694435180363
cano_lastlocdt2: 2.833297762273427
cano_locdt_skew: 2.4517587319708967
stocn_value_counts_cano: 2.326414494648168
scity: 2.2629618285926467
kmeans_all_feature: 2.2293984354063214
stocn_value_counts_all: 2.1371176161903427
hcefg_value_counts_cano: 2.1163034951168114
bacno_cano_nunique: 2.0536793685522854
stscd_value_counts_cano: 1.6842300453760997
stocn_value_counts: 1.5534921786441682
bacno_locdt_skew: 1.5497727979392995
stocn: 1.51521216787127
pca3_feature_1: 1.4499007193801348
hcefg_ratio_cano: 1.2600933930655378
conam_log: 1.2476780350995746
conam: 1.069228969834466
kmeans_all_feature5: 1.0538834410193687
csmcu: 0.9876464247458699
cano_conam_kurt: 0.8972066433595902
cano_conam_skew: 0.8832757584826938
cano_mchno_mode: 0.8552273052302565
isol

In [83]:
# LossFunctionChange
feature_importances = model.get_feature_importance(validation_pool,type="LossFunctionChange")
print('LossFunctionChange')
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

LossFunctionChange
stocn_cano_nunique: 0.0029041971351163216
bacno_1_count: 0.0012787351816356367
stscd: 0.000879872964887958
stocn_bacno_nunique: 0.0008518063620501434
scity: 0.0005837563327632002
stscd_ratio_cano: 0.0004599980792235961
ecfg: 0.00037785417362901433
stocn: 0.0003707865636862172
stocn_value_counts_all: 0.000331018550505182
isolationtree_all_feature3: 0.00025912105007412645
conam_log: 0.00022309440517957052
cano_locdt_skew: 0.00021879624987027058
stocn_value_counts: 0.00021860033692473746
conam: 0.0002026718210068508
bacno_cano_monoincrease: 0.0001372749809386356
cano_lastlocdt2: 0.00013179657699450772
bacno_loctm_diff: 0.00012098570075934345
bacno_3day_count: 8.132228469901956e-05
mchno_cano_nunique: 6.790787177825622e-05
isolationtree_all_feature4: 6.690065107308601e-05
bacno_2day_count: 6.61757648207778e-05
stscd_ratio: 6.468870840510912e-05
mcc_bacno_nunique: 6.44358814408802e-05
bacno_7day_count: 5.5056929691345985e-05
bacno_consecutive_and_only_ecfg: 5.496516589362

In [84]:
# Shapvalue
feature_importances = model.get_feature_importance(validation_pool,type="ShapValues")
print('Shapvalue')
feature_importances = np.mean(abs(feature_importances),axis=0)
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Shapvalue
isolationtree_all_feature3: 0.32986393303379713
stocn_cano_nunique: 0.29787278044354054
ecfg: 0.207077845401178
bacno_1_count: 0.1835013154168661
stocn_bacno_nunique: 0.15536283051156383
cano_lastlocdt2: 0.1531568275879238
bacno_cano_nunique: 0.15124997510425595
cano_locdt_skew: 0.144731834694322
contp_value_counts_cano: 0.1144800594368656
stocn_value_counts: 0.11404410043422071
stscd_value_counts_cano: 0.10065513952925702
stocn_value_counts_all: 0.09498325027270813
bacno_locdt_skew: 0.09349494654462519
hcefg_value_counts_cano: 0.08905168493066702
bacno_loctm_diff: 0.08301403176098411
scity: 0.07788547511849551
bacno_cano_monoincrease: 0.06551110708232687
kmeans_all_feature5: 0.06447223283168922
cano_txkey_nunique: 0.06289992249568226
stocn: 0.06226880881978652
mchno_value_counts_cano: 0.05448249965583049
hcefg_ratio_cano: 0.05328248908564928
kmeans_all_feature: 0.05318824592054904
cano_conam_skew: 0.0523607801791621
stscd_ratio_cano: 0.05044059469238794
pca3_feature_1: 0.048

In [86]:
res = model.calc_feature_statistics(X_train1,
                                    y_train1,
                                    prediction_type='Probability',
#                                     cat_feature_values={6189:0}, 
                                    max_cat_features_on_plot=20,
                                    feature=['mchno'],#["mchno"],
                                    plot=True)

In [94]:
valid_all_data = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)]

In [95]:
tmp_df = valid_all_data.groupby(['bacno'])['fraud_ind'].apply(lambda s: s[s==1].shape[0])
print(tmp_df.value_counts())

0     77014
1      1330
2       590
3       232
4       109
5        43
6        23
7        18
8        18
9        17
10        6
18        5
13        5
11        5
15        4
16        3
12        2
22        2
25        2
32        1
14        1
82        1
19        1
21        1
30        1
37        1
Name: fraud_ind, dtype: int64


In [96]:
valid_all_data_correct = valid_all_data[valid_all_data['fraud_ind']==y_test1_pred]
print(valid_all_data_correct.shape)
print(valid_all_data.shape)

(502900, 258)
(506996, 258)


In [103]:
valid_all_data['correct'] = valid_all_data['fraud_ind']==y_test1_pred

/home/kaiwen/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [97]:
tmp_df2 = valid_all_data_correct.groupby(['bacno'])['fraud_ind'].apply(lambda s: s[s==1].shape[0])
print(tmp_df2.value_counts())

0     77821
1       826
2       422
3       137
4        43
5        21
7         5
6         5
9         4
8         2
30        1
19        1
18        1
16        1
12        1
10        1
Name: fraud_ind, dtype: int64


In [99]:
bacno_fraud_count = valid_all_data.groupby('bacno').apply(lambda s:s[s['fraud_ind']==1]['bacno'].shape[0])

In [105]:
list1=['locdt','cano','etymd','mchno','acqic','mcc','stocn','conam','correct','fraud_ind']

list2=['locdt','cano','cano_lastlocdt2','contp','csmcu','correct','fraud_ind',\
       'flbmk','flg_3dsmk','hcefg','insfg','iterm','mchno','ovrlt','correct','fraud_ind',\
       'scity','stocn','stscd','conam','etymd','mcc','ecfg','correct','fraud_ind']

pd.set_option('display.max_columns', 500)
bacno_10fraud_more_list = bacno_fraud_count[(bacno_fraud_count>=2)].index
pd.set_option('display.max_rows', 1000)

cnt=0
for b in bacno_10fraud_more_list:
    cnt+=1
    if cnt>100:
        break
    print()
    
    print(b,valid_all_data[valid_all_data['bacno']==b]['fraud_ind'].sum())
    print(valid_all_data[valid_all_data['bacno']==b][list2])


35 2.0
     locdt   cano  cano_lastlocdt2  contp  csmcu  correct  fraud_ind  flbmk  \
366     61  99027                0      5     62     True        0.0    0.0   
367     64  99027                0      5     62     True        0.0    0.0   
368     64  99027                0      5     61     True        1.0    0.0   
369     64  99027                0      5     61     True        1.0    0.0   
370     65  99027                0      5     62     True        0.0    0.0   
371     65  99027                0      5     62     True        0.0    0.0   
372     65  99027                0      5     61     True        0.0    0.0   
373     65  99027                0      5     61     True        0.0    0.0   
374     76  99027                0      5     61     True        0.0    0.0   
375     76  99027                0      5     61     True        0.0    0.0   

     flg_3dsmk  hcefg  insfg  iterm  mchno  ovrlt  correct  fraud_ind  scity  \
366        0.0      5      0      0  82174

       locdt   cano  cano_lastlocdt2  contp  csmcu  correct  fraud_ind  flbmk  \
22233     62  87559                0      5     62     True        0.0    0.0   
22234     62  87559                0      5     62     True        0.0    0.0   
22235     63  87559                0      5     62     True        0.0    0.0   
22236     65  87559                0      5     62     True        0.0    0.0   
22237     65  87559                0      5     62     True        0.0    0.0   
22238     65  87559                0      5     62     True        0.0    0.0   
22239     66  87559                0      5     62     True        0.0    0.0   
22240     69  87559                0      5     62     True        0.0    0.0   
22241     69  87559                0      5     62     True        0.0    0.0   
22242     70  87559                0      4     62     True        0.0    0.0   
22243     70  87559                0      5     62     True        0.0    0.0   
22244     70  87559         

       locdt   cano  cano_lastlocdt2  contp  csmcu  correct  fraud_ind  flbmk  \
32669     76  31683                0      5      0     True        0.0    0.0   
32670     78  31684                0      5     62    False        1.0    0.0   
32671     78  31684                0      5     62    False        1.0    0.0   
32672     78  31684                0      5     62    False        1.0    0.0   
32673     78  31684                0      5     62    False        1.0    0.0   
32674     78  31684                0      5     62     True        0.0    0.0   
32675     81  31683                0      5      0     True        0.0    0.0   
32676     83  31683                0      5      0     True        0.0    0.0   
32677     83  31684                0      5     62    False        1.0    0.0   
32678     86  31683                0      5      0     True        0.0    0.0   
32679     88  31684                0      5     62    False        1.0    0.0   
32680     89  31683         

       locdt    cano  cano_lastlocdt2  contp  csmcu  correct  fraud_ind  \
56715     74  156001                0      2     62     True        0.0   
56716     76  156001                0      5     62     True        0.0   
56717     88  156001                0      5     62     True        0.0   
56718     88  156001                0      5     62    False        1.0   
56719     89  156001                0      5     62     True        0.0   
56720     89  156001                0      5     62    False        1.0   

       flbmk  flg_3dsmk  hcefg  insfg  iterm  mchno  ovrlt  correct  \
56715    0.0        0.0      5      0      0  93910      0     True   
56716    0.0        0.0      5      0      0  78071      0     True   
56717    0.0        0.0      5      0      0  54238      0     True   
56718    0.0        0.0      5      0      0  54238      0    False   
56719    0.0        0.0      5      0      0  93841      0     True   
56720    0.0        0.0      5      0      0  54

       locdt   cano  cano_lastlocdt2  contp  csmcu  correct  fraud_ind  flbmk  \
68367     63  44388                0      5     62     True        0.0    0.0   
68368     67  44388                0      5     62     True        0.0    0.0   
68369     78  44388                0      5     62     True        0.0    0.0   
68370     80  44390                1      5     62     True        0.0    0.0   
68371     82  44389                1      5     62     True        0.0    0.0   
68372     82  44389                1      5     62     True        0.0    0.0   
68373     82  44389                1      5     62     True        0.0    0.0   
68374     84  44388                0      5     62     True        0.0    0.0   
68375     86  44388                0      5     62     True        0.0    0.0   
68376     88  44388                0      5      4     True        0.0    0.0   
68377     89  44388                0      5      4    False        1.0    0.0   
68378     89  44388         

       locdt    cano  cano_lastlocdt2  contp  csmcu  correct  fraud_ind  \
87537     76  204415                0      5     62     True        1.0   
87538     87  204415                0      5     62     True        1.0   
87539     87  204415                0      5     62     True        1.0   

       flbmk  flg_3dsmk  hcefg  insfg  iterm  mchno  ovrlt  correct  \
87537    0.0        0.0      5      0      0  54238      0     True   
87538    0.0        0.0      5      0      0  54238      0     True   
87539    0.0        0.0      5      0      0  54238      0     True   

       fraud_ind  scity  stocn  stscd    conam  etymd  mcc  ecfg  correct  \
87537        1.0   6580     38      0     0.00      8  380     1     True   
87538        1.0   6580     38      2  1016.34      8  380     1     True   
87539        1.0   6580     38      2  1016.34      8  380     1     True   

       fraud_ind  
87537        1.0  
87538        1.0  
87539        1.0  

9631 2.0
       locdt   cano

        locdt   cano  cano_lastlocdt2  contp  csmcu  correct  fraud_ind  \
113932     61  76789                0      5     62     True        0.0   
113933     61  76789                0      5     62    False        0.0   
113934     61  76789                0      5     62     True        1.0   
113935     61  76789                0      5     62     True        1.0   
113936     61  76788                0      5     62     True        0.0   

        flbmk  flg_3dsmk  hcefg  insfg  iterm  mchno  ovrlt  correct  \
113932    0.0        0.0      5      0      0  54238      0     True   
113933    0.0        0.0      5      0      0  54238      0    False   
113934    0.0        0.0      5      0      0  54238      0     True   
113935    0.0        0.0      5      0      0  54238      0     True   
113936    0.0        0.0      5      0      0  54828      0     True   

        fraud_ind  scity  stocn  stscd    conam  etymd  mcc  ecfg  correct  \
113932        0.0   6580     38      0

        locdt    cano  cano_lastlocdt2  contp  csmcu  correct  fraud_ind  \
134903     64  103907                0      5      0     True        0.0   
134904     72  103907                0      5     62    False        1.0   
134905     72  103907                0      5     62    False        1.0   
134906     73  103907                1      5     62     True        1.0   
134907     73  103907                1      5     62     True        1.0   
134908     73  103907                1      5     62     True        1.0   
134909     73  103907                1      5     62     True        1.0   
134910     84  103908                0      5      0     True        0.0   
134911     87  103908                0      5     62     True        0.0   
134912     88  103908                0      5      0     True        0.0   

        flbmk  flg_3dsmk  hcefg  insfg  iterm  mchno  ovrlt  correct  \
134903    0.0        0.0      5      0      0  59333      0     True   
134904    0.0      

In [87]:
## 理論上th設0.5一定是最好的？

y_test1_pred = model.predict_proba(X_test1,verbose=True)[:,1]
th=0.37

y_test1_pred[y_test1_pred>th]=1
y_test1_pred[y_test1_pred<=th]=0
print('F1 score',f1_score(y_test1, y_test1_pred))

tn, fp, fn, tp = confusion_matrix(y_test1, y_test1_pred).ravel()
print('tn fp fn tp')
print(tn, fp, fn, tp)
print('Percision', tp/(tp+fp))
print('Recall',tp/(tp+fn))

F1 score 0.5574762316335349
tn fp fn tp
500320 1527 2569 2580
Percision 0.6281957633308984
Recall 0.5010681685764226


In [21]:
model = CatBoostClassifier(**param_cat)
model.fit(
    X_train_all, y_train_all,
    cat_features=categorical_features_indices,    
    silent=False
)

0:	learn: 0.4601475	total: 1.34s	remaining: 55m 49s
1:	learn: 0.4452679	total: 2.9s	remaining: 1h 22s
2:	learn: 0.4464137	total: 4.43s	remaining: 1h 1m 26s
3:	learn: 0.4579100	total: 5.98s	remaining: 1h 2m 13s
4:	learn: 0.5168985	total: 7.54s	remaining: 1h 2m 42s
5:	learn: 0.5087237	total: 9.19s	remaining: 1h 3m 38s
6:	learn: 0.5314942	total: 10.6s	remaining: 1h 3m 6s
7:	learn: 0.5365445	total: 11.9s	remaining: 1h 2m 1s
8:	learn: 0.5412788	total: 13.4s	remaining: 1h 1m 55s
9:	learn: 0.5337798	total: 15s	remaining: 1h 2m 26s
10:	learn: 0.5386611	total: 16.5s	remaining: 1h 2m 16s
11:	learn: 0.5319812	total: 17.9s	remaining: 1h 1m 50s
12:	learn: 0.5395877	total: 19.3s	remaining: 1h 1m 23s
13:	learn: 0.5405121	total: 20.6s	remaining: 1h 52s
14:	learn: 0.5512883	total: 21.9s	remaining: 1h 29s
15:	learn: 0.5588370	total: 23.3s	remaining: 1h 24s
16:	learn: 0.5664056	total: 24.7s	remaining: 1h 3s
17:	learn: 0.5762766	total: 25.9s	remaining: 59m 35s
18:	learn: 0.5821506	total: 27.3s	remaining: 

153:	learn: 0.7461836	total: 3m 43s	remaining: 56m 40s
154:	learn: 0.7471765	total: 3m 44s	remaining: 56m 38s
155:	learn: 0.7472059	total: 3m 45s	remaining: 56m 33s
156:	learn: 0.7476752	total: 3m 47s	remaining: 56m 31s
157:	learn: 0.7481728	total: 3m 48s	remaining: 56m 27s
158:	learn: 0.7482575	total: 3m 49s	remaining: 56m 25s
159:	learn: 0.7485640	total: 3m 51s	remaining: 56m 24s
160:	learn: 0.7488982	total: 3m 52s	remaining: 56m 24s
161:	learn: 0.7489267	total: 3m 54s	remaining: 56m 21s
162:	learn: 0.7488200	total: 3m 55s	remaining: 56m 19s
163:	learn: 0.7492110	total: 3m 57s	remaining: 56m 18s
164:	learn: 0.7494314	total: 3m 58s	remaining: 56m 15s
165:	learn: 0.7496520	total: 4m	remaining: 56m 15s
166:	learn: 0.7505466	total: 4m 1s	remaining: 56m 13s
167:	learn: 0.7506247	total: 4m 2s	remaining: 56m 11s
168:	learn: 0.7510572	total: 4m 4s	remaining: 56m 10s
169:	learn: 0.7511918	total: 4m 5s	remaining: 56m 7s
170:	learn: 0.7516101	total: 4m 7s	remaining: 56m 5s
171:	learn: 0.7517095

304:	learn: 0.7793990	total: 7m 16s	remaining: 52m 23s
305:	learn: 0.7792070	total: 7m 17s	remaining: 52m 20s
306:	learn: 0.7792751	total: 7m 19s	remaining: 52m 17s
307:	learn: 0.7797282	total: 7m 20s	remaining: 52m 15s
308:	learn: 0.7803286	total: 7m 21s	remaining: 52m 13s
309:	learn: 0.7804769	total: 7m 23s	remaining: 52m 11s
310:	learn: 0.7808021	total: 7m 24s	remaining: 52m 10s
311:	learn: 0.7808918	total: 7m 26s	remaining: 52m 10s
312:	learn: 0.7808361	total: 7m 27s	remaining: 52m 9s
313:	learn: 0.7809912	total: 7m 29s	remaining: 52m 7s
314:	learn: 0.7810783	total: 7m 30s	remaining: 52m 4s
315:	learn: 0.7814182	total: 7m 31s	remaining: 52m 2s
316:	learn: 0.7814835	total: 7m 33s	remaining: 52m 1s
317:	learn: 0.7818329	total: 7m 34s	remaining: 51m 59s
318:	learn: 0.7817650	total: 7m 36s	remaining: 51m 58s
319:	learn: 0.7819373	total: 7m 37s	remaining: 51m 55s
320:	learn: 0.7824778	total: 7m 38s	remaining: 51m 53s
321:	learn: 0.7827371	total: 7m 39s	remaining: 51m 50s
322:	learn: 0.7

454:	learn: 0.8014783	total: 10m 47s	remaining: 48m 28s
455:	learn: 0.8017645	total: 10m 48s	remaining: 48m 27s
456:	learn: 0.8017424	total: 10m 49s	remaining: 48m 25s
457:	learn: 0.8019630	total: 10m 51s	remaining: 48m 25s
458:	learn: 0.8018199	total: 10m 53s	remaining: 48m 23s
459:	learn: 0.8020181	total: 10m 54s	remaining: 48m 23s
460:	learn: 0.8021169	total: 10m 56s	remaining: 48m 21s
461:	learn: 0.8019627	total: 10m 57s	remaining: 48m 19s
462:	learn: 0.8022266	total: 10m 58s	remaining: 48m 18s
463:	learn: 0.8023038	total: 11m	remaining: 48m 17s
464:	learn: 0.8023922	total: 11m 1s	remaining: 48m 16s
465:	learn: 0.8024691	total: 11m 3s	remaining: 48m 14s
466:	learn: 0.8027998	total: 11m 4s	remaining: 48m 12s
467:	learn: 0.8026348	total: 11m 5s	remaining: 48m 11s
468:	learn: 0.8027125	total: 11m 7s	remaining: 48m 9s
469:	learn: 0.8026461	total: 11m 8s	remaining: 48m 7s
470:	learn: 0.8027788	total: 11m 9s	remaining: 48m 5s
471:	learn: 0.8030320	total: 11m 11s	remaining: 48m 4s
472:	le

602:	learn: 0.8163746	total: 14m 13s	remaining: 44m 46s
603:	learn: 0.8164517	total: 14m 15s	remaining: 44m 44s
604:	learn: 0.8165991	total: 14m 16s	remaining: 44m 42s
605:	learn: 0.8168204	total: 14m 17s	remaining: 44m 41s
606:	learn: 0.8169800	total: 14m 19s	remaining: 44m 39s
607:	learn: 0.8172337	total: 14m 20s	remaining: 44m 38s
608:	learn: 0.8171713	total: 14m 22s	remaining: 44m 37s
609:	learn: 0.8173132	total: 14m 23s	remaining: 44m 35s
610:	learn: 0.8173008	total: 14m 25s	remaining: 44m 34s
611:	learn: 0.8177165	total: 14m 26s	remaining: 44m 33s
612:	learn: 0.8179330	total: 14m 27s	remaining: 44m 31s
613:	learn: 0.8181395	total: 14m 29s	remaining: 44m 29s
614:	learn: 0.8180873	total: 14m 30s	remaining: 44m 28s
615:	learn: 0.8181544	total: 14m 32s	remaining: 44m 27s
616:	learn: 0.8184705	total: 14m 33s	remaining: 44m 26s
617:	learn: 0.8184157	total: 14m 35s	remaining: 44m 25s
618:	learn: 0.8185575	total: 14m 36s	remaining: 44m 23s
619:	learn: 0.8183485	total: 14m 37s	remaining: 

...

1770:	learn: 0.8864206	total: 42m 9s	remaining: 17m 21s
1771:	learn: 0.8864794	total: 42m 10s	remaining: 17m 19s
1772:	learn: 0.8865669	total: 42m 11s	remaining: 17m 18s
1773:	learn: 0.8865669	total: 42m 13s	remaining: 17m 16s
1774:	learn: 0.8866023	total: 42m 14s	remaining: 17m 15s
1775:	learn: 0.8866610	total: 42m 16s	remaining: 17m 13s
1776:	learn: 0.8870368	total: 42m 17s	remaining: 17m 12s
1777:	learn: 0.8870781	total: 42m 18s	remaining: 17m 10s
1778:	learn: 0.8871189	total: 42m 20s	remaining: 17m 9s
1779:	learn: 0.8870776	total: 42m 21s	remaining: 17m 8s
1780:	learn: 0.8871716	total: 42m 23s	remaining: 17m 6s
1781:	learn: 0.8872184	total: 42m 25s	remaining: 17m 5s
1782:	learn: 0.8872244	total: 42m 26s	remaining: 17m 4s
1783:	learn: 0.8872478	total: 42m 27s	remaining: 17m 2s
1784:	learn: 0.8873239	total: 42m 29s	remaining: 17m 1s
1785:	learn: 0.8872946	total: 42m 30s	remaining: 16m 59s
1786:	learn: 0.8874529	total: 42m 32s	remaining: 16m 58s
1787:	learn: 0.8874120	total: 42m 33s	r

1915:	learn: 0.8918166	total: 45m 37s	remaining: 13m 54s
1916:	learn: 0.8920674	total: 45m 38s	remaining: 13m 52s
1917:	learn: 0.8920383	total: 45m 39s	remaining: 13m 51s
1918:	learn: 0.8920326	total: 45m 41s	remaining: 13m 49s
1919:	learn: 0.8920965	total: 45m 42s	remaining: 13m 48s
1920:	learn: 0.8921313	total: 45m 44s	remaining: 13m 47s
1921:	learn: 0.8925107	total: 45m 45s	remaining: 13m 45s
1922:	learn: 0.8925924	total: 45m 47s	remaining: 13m 44s
1923:	learn: 0.8926215	total: 45m 49s	remaining: 13m 43s
1924:	learn: 0.8925568	total: 45m 50s	remaining: 13m 41s
1925:	learn: 0.8928187	total: 45m 52s	remaining: 13m 40s
1926:	learn: 0.8928478	total: 45m 53s	remaining: 13m 38s
1927:	learn: 0.8929472	total: 45m 55s	remaining: 13m 37s
1928:	learn: 0.8929472	total: 45m 56s	remaining: 13m 35s
1929:	learn: 0.8929125	total: 45m 58s	remaining: 13m 34s
1930:	learn: 0.8930345	total: 45m 59s	remaining: 13m 33s
1931:	learn: 0.8930692	total: 46m 1s	remaining: 13m 31s
1932:	learn: 0.8930927	total: 46

2142:	learn: 0.9000681	total: 51m 4s	remaining: 8m 30s
2143:	learn: 0.9000628	total: 51m 5s	remaining: 8m 29s
2144:	learn: 0.9000393	total: 51m 7s	remaining: 8m 27s
2145:	learn: 0.8999581	total: 51m 8s	remaining: 8m 26s
2146:	learn: 0.9000497	total: 51m 10s	remaining: 8m 24s
2147:	learn: 0.8999581	total: 51m 11s	remaining: 8m 23s
2148:	learn: 0.9000157	total: 51m 13s	remaining: 8m 21s
2149:	learn: 0.9000968	total: 51m 14s	remaining: 8m 20s
2150:	learn: 0.9001204	total: 51m 16s	remaining: 8m 19s
2151:	learn: 0.9002146	total: 51m 17s	remaining: 8m 17s
2152:	learn: 0.9003769	total: 51m 19s	remaining: 8m 16s
2153:	learn: 0.9004528	total: 51m 20s	remaining: 8m 14s
2154:	learn: 0.9005340	total: 51m 22s	remaining: 8m 13s
2155:	learn: 0.9005340	total: 51m 23s	remaining: 8m 11s
2156:	learn: 0.9005104	total: 51m 24s	remaining: 8m 10s
2157:	learn: 0.9004816	total: 51m 26s	remaining: 8m 9s
2158:	learn: 0.9005392	total: 51m 27s	remaining: 8m 7s
2159:	learn: 0.9004372	total: 51m 29s	remaining: 8m 6s

2290:	learn: 0.9049485	total: 54m 38s	remaining: 4m 59s
2291:	learn: 0.9050679	total: 54m 40s	remaining: 4m 57s
2292:	learn: 0.9051251	total: 54m 42s	remaining: 4m 56s
2293:	learn: 0.9051301	total: 54m 43s	remaining: 4m 54s
2294:	learn: 0.9051015	total: 54m 44s	remaining: 4m 53s
2295:	learn: 0.9050157	total: 54m 46s	remaining: 4m 51s
2296:	learn: 0.9049920	total: 54m 47s	remaining: 4m 50s
2297:	learn: 0.9050393	total: 54m 49s	remaining: 4m 49s
2298:	learn: 0.9050344	total: 54m 50s	remaining: 4m 47s
2299:	learn: 0.9050294	total: 54m 52s	remaining: 4m 46s
2300:	learn: 0.9050008	total: 54m 53s	remaining: 4m 44s
2301:	learn: 0.9051053	total: 54m 54s	remaining: 4m 43s
2302:	learn: 0.9050954	total: 54m 56s	remaining: 4m 41s
2303:	learn: 0.9049958	total: 54m 57s	remaining: 4m 40s
2304:	learn: 0.9050344	total: 54m 59s	remaining: 4m 39s
2305:	learn: 0.9049535	total: 55m	remaining: 4m 37s
2306:	learn: 0.9050630	total: 55m 2s	remaining: 4m 36s
2307:	learn: 0.9049485	total: 55m 3s	remaining: 4m 34

2438:	learn: 0.9092473	total: 58m 10s	remaining: 1m 27s
2439:	learn: 0.9092852	total: 58m 11s	remaining: 1m 25s
2440:	learn: 0.9092900	total: 58m 13s	remaining: 1m 24s
2441:	learn: 0.9093563	total: 58m 14s	remaining: 1m 23s
2442:	learn: 0.9094038	total: 58m 16s	remaining: 1m 21s
2443:	learn: 0.9094653	total: 58m 17s	remaining: 1m 20s
2444:	learn: 0.9094985	total: 58m 19s	remaining: 1m 18s
2445:	learn: 0.9095601	total: 58m 20s	remaining: 1m 17s
2446:	learn: 0.9095269	total: 58m 22s	remaining: 1m 15s
2447:	learn: 0.9095128	total: 58m 23s	remaining: 1m 14s
2448:	learn: 0.9095696	total: 58m 25s	remaining: 1m 12s
2449:	learn: 0.9096312	total: 58m 26s	remaining: 1m 11s
2450:	learn: 0.9097639	total: 58m 28s	remaining: 1m 10s
2451:	learn: 0.9097401	total: 58m 29s	remaining: 1m 8s
2452:	learn: 0.9097117	total: 58m 31s	remaining: 1m 7s
2453:	learn: 0.9097117	total: 58m 32s	remaining: 1m 5s
2454:	learn: 0.9097686	total: 58m 34s	remaining: 1m 4s
2455:	learn: 0.9097117	total: 58m 35s	remaining: 1m 

In [25]:
y_test_pred_cat = model.predict_proba(X_test_all)[:,1]

print(X_test_all.index)

th=0.35
y_test_pred_cat[y_test_pred_cat>th]=1
y_test_pred_cat[y_test_pred_cat<=th]=0

Int64Index([1521787, 1521788, 1521789, 1521790, 1521791, 1521792, 1521793,
            1521794, 1521795, 1521796,
            ...
            1943442, 1943443, 1943444, 1943445, 1943446, 1943447, 1943448,
            1943449, 1943450, 1943451],
           dtype='int64', length=421665)


## write csv

In [26]:
import time
print(time.localtime( time.time() ))
a = time.localtime( time.time() )
print(str(a.tm_mon)+str(a.tm_mday))

time.struct_time(tm_year=2019, tm_mon=10, tm_mday=31, tm_hour=1, tm_min=8, tm_sec=37, tm_wday=3, tm_yday=304, tm_isdst=0)
1031


In [27]:
result = y_test_pred_cat
test_data_txkey = all_data[all_data['locdt']>90]['txkey'].values

print('{}: prediction positive ratio'.format(result.sum()/result.shape[0]))
print('{}: training positive ratio'.format(y_train_all.sum()/y_train_all.shape[0]))

import time
t_now = time.localtime( time.time() )
t = str(t_now.tm_mon)+str(t_now.tm_mday)+str(t_now.tm_hour)+str(t_now.tm_min)
print('Now:',t)

submit_file_name='submit_cat_AN3_over5_th{}_time{}.csv'.format(th,t)
import csv
with open('../../prediction/{}'.format(submit_file_name),'w') as f:
    writer = csv.writer(f)
    writer.writerow(['txkey','fraud_ind'])
    for i in range(result.shape[0]):
        writer.writerow([test_data_txkey[i], result[i]])
        
# with open('../prediction/log.txt','w') as f:
#     print('{}'.format(submit_file_name),file=f)
#     print('delete_list:\n{}'.format(delete_list),file=f)

0.014274364720809174: prediction positive ratio
0.013375722095142093: training positive ratio
Now: 103118


In [ ]:
## 用不同seed訓練 降低variance

predictions = []

for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.1, iterations=1000, random_seed=i, logging_level='Silent')
    clf.fit(train_df, labels, cat_features=cat_features_ids)
    predictions.append(clf.predict_proba(test_df)[:,1])
    

prediction = np.mean(predictions, axis=0)

pd.DataFrame(
    {'ID':test_id, 'PredictedProb':prediction}
).to_csv(
    'submission_improved_bagged.csv', index=False
)

## 贝叶斯调参
* http://nohup.cc/article/258/
* https://github.com/fmfn/BayesianOptimization

Results:
{'params': {'bagging_temperature': 1.0,
  'learning_rate': 0.2,
  'reg_lambda': 17.267046492047776},
 'target': 0.5339179149920837}

In [ ]:
# from bayes_opt import BayesianOptimization
# def cat_train(bagging_temperature, reg_lambda, learning_rate):
#     params = {
#         'iterations':2000,
#         'depth':6,
#         'bagging_temperature':bagging_temperature,
#         'reg_lambda':reg_lambda,
#         'learning_rate':learning_rate,
#         'loss_function':'Logloss',
#         'eval_metric':'F1',
#         'random_seed':random_seed,
#         'verbose':30
#     }
 
#     model = CatBoostClassifier(**params)
#     # 评价数据集是验证集，评价指标是AUC
#     model.fit(X_train1, y_train1,\
#               eval_set=(X_test1, y_test1),\
#               cat_features=categorical_features_indices,\
#               early_stopping_rounds=200) 
     
#     print(params)
#     score_max = model.best_score_.get('validation').get('F1')
#     return score_max
 
# cat_opt = BayesianOptimization(cat_train, 
#                            {
#                               'bagging_temperature': (1, 50),  
#                               'reg_lambda': (1, 200),
#                               'learning_rate':(0.05, 0.2)
#                             })
 
# cat_opt.maximize(n_iter=15, init_points=random_seed)
# cat_opt.max

## Permutation importance

In [ ]:
# def permutation_scoring(model,X,y):
#     y_pred = model.predict(X)
#     return f1_score(y, y_pred)

# import eli5
# from eli5.sklearn import PermutationImportance

# print(y_test1.sum())
# print(model.score(X_test1, y_test1))
# perm = PermutationImportance(model, random_state=random_seed,scoring=permutation_scoring,n_iter=50).fit(X_test1, y_test1)
# feature_importance1 = pd.DataFrame({'feature':X_test1.columns.tolist(),'importance':perm.feature_importances_})
# delete_col1 = feature_importance1.iloc[:,0][(feature_importance1['importance'].values)<=0.0000]
# print(delete_col1)

# feature_importance1.sort_values(by=['importance'],inplace=True)
# top_col = feature_importance1
# for i in range(top_col.shape[0]):
#     print(top_col.iloc[i].values)

In [ ]:
# from eli5.catboost import explain_weights_catboost
# explain_weights_catboost(model, vec=None, top=70, importance_type='PredictionValuesChange', feature_names=None, pool=None)

## 將使用者分類來訓練模型:
1. 根據cano的個數分類
2. 根據txkey的個數分類
3. 根據stocn的眾數分類

murmur: 感覺這方法怪怪的,tree模型應該就能涵蓋進去

In [ ]:
## txkey
txkey_qcut_id = pd.qcut(all_data['bacno_txkey_nunique'],3,labels=[0,1,2])

# print(all_data['bacno_txkey_nunique'])
# print(txkey_qcut_id)
models=[]
for i in range(3):
    print(i)

## 不同model

In [ ]:
def th_transform(th,y_test1_pred,y_test1):
    y_test1_pred[y_test1_pred>th]=1
    y_test1_pred[y_test1_pred<=th]=0
    print('F1 score',f1_score(y_test1, y_test1_pred))
    
    return y_test1_pred

In [ ]:
rf = RandomForestClassifier(n_estimators=200,random_state=random_seed,verbose=0,class_weight="balanced", criterion='gini')

rf.fit(X_train1, y_train1) 
y_pred_rf = rf.predict_proba(X_test1)[:,1]
th_transform(0.37,y_pred_rf,y_test1)

In [ ]:
def lgb_f1_score(y_true, y_pred):
    y_pred = np.round(y_pred) # scikits f1 doesn't like probabilities
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print()
    print('tn, fp, fn, tp')
    print(tn, fp, fn, tp)
    return 'f1', f1_score(y_true, y_pred), True

param_dist_lgb = {
                  'num_leaves':26, 
#                   'max_depth':-1, 
                  'learning_rate':0.1, 
                  'n_estimators':2000,
                  'objective': 'binary',
#                   'subsample': 1, 
#                   'colsample_bytree': 0.5, 
                  'lambda_l1': 10,
                  'lambda_l2': 10,
#                   'min_child_weight': 1,
                  'random_state': random_seed,
                 }
evals_result = {}

lgb_clf = LGBMClassifier(**param_dist_lgb)
lgb_clf.fit(X_train1, y_train1,
        eval_set=[(X_train1, y_train1),(X_test1, y_test1)],
        eval_metric=lgb_f1_score,
        early_stopping_rounds=50,
        verbose=True,
        callbacks=[lgb.record_evaluation(evals_result)]
        )
y_test_pred = lgb_clf.predict(X_test1)
print('F1 score',f1_score(y_test1, y_test_pred))

## Anomaly detection
* one class svm
* isolation tree
* replicator NN
* Kmeans?
* KNN(take too much time)

## 異常偵測
wiki<br>
https://zh.wikipedia.org/wiki/%E5%BC%82%E5%B8%B8%E6%A3%80%E6%B5%8B#cite_note-9

因為盜刷很可能都是outlier，一般的機器學習方法在outlier上表現會很差，因此可以用來解釋為什麼會train不好的原因
http://www.cainiaoxueyuan.com/suanfa/7017.html<br>
https://towardsdatascience.com/outlier-detection-with-isolation-forest-3d190448d45e<br>
https://medium.com/@cyeninesky3/oneclass-svm-%E7%95%B0%E5%B8%B8%E6%AA%A2%E6%B8%AC%E4%BB%BB%E5%8B%99-anomaly-detection-%E7%9A%84%E7%AE%97%E6%B3%95%E7%90%86%E8%A7%A3%E8%88%87%E5%AF%A6%E8%B8%90-cf5f0bbb01c0<br>

isolation tree<br>
https://zhuanlan.zhihu.com/p/25040651
https://scikit-learn.org/stable/auto_examples/ensemble/plot_isolation_forest.html
https://towardsdatascience.com/outlier-detection-with-extended-isolation-forest-1e248a3fe97b

oneclass svm<br>
https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html

Replicator NN<br>
https://togaware.com/papers/dawak02.pdf

one class kmeans?<br>
https://ai100-2.cupoy.com/mission/D57

## 製作特徵
XGB, LGB, PCA, Isolation Forest, Kmean距離？, oneclass SVM?
當作新feature

In [ ]:
# import xgboost as xgb
# param_dist_xgb = {'learning_rate':0.01, #默认0.3
#               'n_estimators':1000, #树的个数
# #               'max_depth':5,
# #               'min_child_weight':1,
# #               'gamma':0.2,
# #               'subsample':0.8,
# #               'colsample_bytree':0.8,
# #               'objective': 'binary:logistic', #逻辑回归损失函数
# #               'nthread':4,  #cpu线程数
# #               'scale_pos_weight':1,
#               'seed':random_seed}  #随机种子

# evals_result = {}

# xgb_clf = xgb.XGBClassifier(**param_dist_xgb)
# xgb_clf.fit(X_train, y_train,
#         eval_set=[(X_train, y_train),(X_test, y_test)],
#         eval_metric=lgb_f1_score,
#         early_stopping_rounds=600,
#         verbose=True,
# #         callbacks=[xgb.record_evaluation(evals_result)]
#         )

# print('F1',f1_score(y_test, xgb_clf.predict(X_test)))
# xgb_X_train = xgb_clf.apply(X_train)
# xgb_X_test = xgb_clf.apply(X_test)

## Train on LGB(未調參數)(效果不好)

In [ ]:
def lgb_f1_score(y_true, y_pred):
    y_pred = np.round(y_pred) # scikits f1 doesn't like probabilities
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print()
    print('tn, fp, fn, tp')
    print(tn, fp, fn, tp)
    return 'f1', f1_score(y_true, y_pred), True

param_dist_lgb = {
#                   'num_leaves':45, 
#                   'max_depth':5, 
                  'learning_rate':0.1, 
                  'n_estimators':2000,
                  'objective': 'binary',
#                   'subsample': 1, 
#                   'colsample_bytree': 0.5, 
#                   'lambda_l1': 0.1,
#                   'lambda_l2': 0,
#                   'min_child_weight': 1,
                  'random_state': random_seed,
                 }
evals_result = {}

lgb_clf = LGBMClassifier(**param_dist_lgb)
lgb_clf.fit(X_train1, y_train1,
        eval_set=[(X_train1, y_train1),(X_test1, y_test1)],
        eval_metric=lgb_f1_score,
        early_stopping_rounds=50,
        verbose=True,
        callbacks=[lgb.record_evaluation(evals_result)]
        )
y_test_pred = lgb_clf.predict(X_test1)

In [ ]:
print('Plotting metrics recorded during training...')
ax = lgb.plot_metric(evals_result, metric='f1')
plt.show()

print('Plotting feature importances...')
ax = lgb.plot_importance(lgb_clf, max_num_features=30)
plt.show()

print('Plotting 4th tree...')  # one tree use categorical feature to split
ax = lgb.plot_tree(lgb_clf, tree_index=3, figsize=(15, 15), show_info=['split_gain'])
plt.show()

print('Plotting 4th tree with graphviz...')
graph = lgb.create_tree_digraph(lgb_clf, tree_index=3, name='Tree4')
graph.render(view=True)

In [ ]:
feature_importance = np.stack([X_train1.columns.values,lgb_clf.feature_importances_]).transpose()
feature_importance = pd.DataFrame(feature_importance,columns=['feature_name','importance'])
feature_importance.sort_values(by=['importance'],inplace=True,ascending=False)
print(feature_importance)

## PCA visualization in one person who has fraud data

In [ ]:
from sklearn.decomposition import PCA
def PCA_plot(x,label):
    x = x.drop(columns=delete_list)
    
    ## 應該先轉dummy,標準化,再PCA
#     dummy_list=['contp','etymd','stscd','hcefg']
#     dummy_list2=['stocn','scity','csmcu']#'mchno','acqic','mcc',
#     x[dummy_list] = x[dummy_list].astype(object)
#     x[dummy_list2] = x[dummy_list2].astype(object)
#     x = pd.get_dummies(x)    
    
    from sklearn.preprocessing import StandardScaler 
    stdsc = StandardScaler() 
    x = stdsc.fit_transform(x)
    print(x.shape,label.sum())

    PCA_model = PCA(n_components=2)
    train_data_pca = PCA_model.fit_transform(x)
    train_data_pca1 = train_data_pca[label==1]
    train_data_pca0 = train_data_pca[label==0]
    
    plt.clf()
    plt.figure(figsize=(10,10))
    plt.scatter(train_data_pca1[:, 0], train_data_pca1[:, 1], c='r',label='fraud transaction',s=100)
    plt.scatter(train_data_pca0[:, 0], train_data_pca0[:, 1], c='b',label='normal transaction',s=3)
    plt.legend()
    plt.show()
    
bacno_hasfraud = all_data[all_data['fraud_ind']==1]['bacno'].unique()
print(bacno_hasfraud.shape[0])
print(all_data[all_data['fraud_ind']==1].shape[0])

for i in range(bacno_hasfraud.shape[0]):
    if all_data[all_data['bacno']==bacno_hasfraud[i]].shape[0]>300:
        print('Ploting PCA on bacno-{}'.format(bacno_hasfraud[i]))
        PCA_plot(all_data[all_data['bacno']==bacno_hasfraud[i]],all_data[all_data['bacno']==bacno_hasfraud[i]]['fraud_ind'])

## TSNE, Kmeans作圖?

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

c_ratio = y_train.sum()/y_train.shape[0]
# fit the model
clf = IsolationForest(behaviour='new', max_samples=0.8, max_features=1,
                      random_state=random_seed, contamination=c_ratio)
clf.fit(X_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

y_pred_test2 = -y_pred_test
y_pred_test2[y_pred_test2==-1]=0
y_pred_test2.sum()

y_pred_train2 = -y_pred_train
y_pred_train2[y_pred_train2==-1]=0
y_pred_train2.sum()

from sklearn.metrics import f1_score
print(f1_score(y_train, y_pred_train2))
print(f1_score(y_test, y_pred_test2))

isolationtree_X_train = clf.score_samples(X_train)
isolationtree_X_test = clf.score_samples(X_test)

print(isolationtree_X_train)

In [ ]:
# cano_lastlocdt
# bacno_lastlocdt
# bacno_cano_nunique



In [ ]:
# bacno_etymd_nunique
# bacno_contp_nunique

In [ ]:
# 用hinge loss(當SVM)

In [ ]:
# X_train['cents']
# encoding data

# GroupKfold
# vanilla KFold